In [6]:
import pandas
import numpy
from scipy.optimize import minimize

In [7]:
motor = 1400 #rpm

table = pandas.read_table("table.txt")
table['ratio'] = table.spindle / motor
table

,spindle,d1,d2,d3,d4,ratio
0,110,45,a,d,184,0.078571
1,190,45,a,c,159,0.135714
2,220,65,b,d,184,0.157143
3,320,45,a,b,136,0.228571
4,380,65,b,c,159,0.271429
5,430,90,c,d,184,0.307143
6,860,65,b,a,117,0.614286
7,1170,90,c,b,136,0.835714
8,1260,99,d,c,159,0.900000
9,1680,90,c,a,117,1.200000


Kiekvieną lentelės eilutę atitinka viena tiesinė lygtis.

$R = \frac{d_1}{d_2} \frac{d_3}{d_4}$

$d_2 = \frac{d_1 d_3}{R d_4}$

$d_2 - \frac{d_1}{R d_4} d_3 = 0$

Fiksuojame D ir sudarome lygčių sistemą su trim nežinomaisiais: B, C, D. Lygčių sistemos matrica:

In [21]:
diameter_d = 50

def diameters(xs):
    return pandas.Series(list(xs) + [diameter_d], index=list('abcd'))

def actual_ratios(ds):
    dsdict = dict(ds)
    return numpy.array([r.d1 / dsdict[r.d2] * dsdict[r.d3] / r.d4 for i, r in table.iterrows()])

def percent_difference(x, y):
    return x / y - 1

def cost(xs):
    speeds = motor * actual_ratios(diameters(xs))
    delta = percent_difference(speeds, table['spindle'])
    delta *= numpy.where(delta > 0, 2, 1)
    cost = numpy.sum(numpy.abs(delta))
    cost += 40 * abs(percent_difference(min(speeds), min(table['spindle'])))
    cost += 70 * abs(max(0, percent_difference(max(speeds), max(table['spindle']))))
    return cost
    
bounds = (diameter_d, 180)
def initial():
    return sorted(numpy.random.uniform(*bounds, 3), reverse=True)
result = minimize(cost, initial(), bounds=(bounds, bounds, bounds))
for i in range(10):
    r = minimize(cost, initial(), bounds=(bounds, bounds, bounds))
    if r.fun < result.fun:
        print(r.fun)
        result = r
    else:
        print('.', end='')
result

13.8477869634
.....13.8476573994
...

      fun: 13.847657399445335
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.11132251, -0.00023128,  0.00151292])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 780
      nit: 30
   status: 0
  success: True
        x: array([ 121.55844156,   99.92036728,   72.27378953])

In [22]:
def show_result(ds):
    ds = numpy.round(pandas.Series(ds, index=list('abcd')))
    print(ds)
    table['actual_ratio'] = actual_ratios(ds)
    table['actual_spindle'] = table['actual_ratio'] * motor
    return table

## Optimizuota

In [23]:
show_result(diameters(result.x))

a    122
b    100
c     72
d     50
dtype: float64


,spindle,d1,d2,d3,d4,ratio,actual_ratio,actual_spindle
0,110,45,a,d,184,0.078571,0.100232,140.324305
1,190,45,a,c,159,0.135714,0.167028,233.838540
2,220,65,b,d,184,0.157143,0.176630,247.282609
3,320,45,a,b,136,0.228571,0.271215,379.701061
4,380,65,b,c,159,0.271429,0.294340,412.075472
5,430,90,c,d,184,0.307143,0.339674,475.543478
6,860,65,b,a,117,0.614286,0.677778,948.888889
7,1170,90,c,b,136,0.835714,0.919118,1286.764706
8,1260,99,d,c,159,0.900000,0.896604,1255.245283
9,1680,90,c,a,117,1.200000,1.303419,1824.786325


## Apvertus variklio skriemulį

In [25]:
show_result([99, 90, 65, 45])

a    99
b    90
c    65
d    45
dtype: int64


,spindle,d1,d2,d3,d4,ratio,actual_ratio,actual_spindle
0,110,45,a,d,184,0.078571,0.111166,155.632411
1,190,45,a,c,159,0.135714,0.185820,260.148656
2,220,65,b,d,184,0.157143,0.176630,247.282609
3,320,45,a,b,136,0.228571,0.300802,421.122995
4,380,65,b,c,159,0.271429,0.295248,413.347310
5,430,90,c,d,184,0.307143,0.338629,474.080268
6,860,65,b,a,117,0.614286,0.611111,855.555556
7,1170,90,c,b,136,0.835714,0.916290,1282.805430
8,1260,99,d,c,159,0.900000,0.899371,1259.119497
9,1680,90,c,a,117,1.200000,1.171598,1640.236686
